In [188]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, load_dataset
import json

In [2]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)
   
device_index = 0

torch_device = 'cuda:' + str(device_index) if torch.cuda.is_available() else 'cpu'

t = torch.cuda.get_device_properties(device_index).total_memory
r = torch.cuda.memory_reserved(device_index)
a = torch.cuda.memory_allocated(device_index)
f = r-a  # free inside reserved
f_ = t-r # free outside reserved

print()
print(torch.cuda.list_gpu_processes(torch_device))

torch_device, t, r, a, f, f_

NVIDIA A100-PCIE-40GB

GPU:1
process    1775501 uses     3338.000 MB GPU memory


('cuda:0', 42409000960, 0, 0, 0, 42409000960)

In [3]:
# List of available models
models = { # name : path (model, dataset)
    "Mistral-7B-v0.1" : "mistralai/Mistral-7B-v0.1",
    "Mistral-7B-Instruct-v0.1" : "mistralai/Mistral-7B-Instruct-v0.1",
    "BioMistral-7B" : "BioMistral/BioMistral-7B",
    "Mathstral-7B" : "mistralai/Mathstral-7b-v0.1",
    "MetaMath-Mistral-7B" : "meta-math/MetaMath-Mistral-7B"
}

datasets = {
    "Math" : "meta-math/MetaMathQA",
    "Bio" : "BioMistral/BioMistralQA", # Dataset doesn't exist (Find another one)
}

In [4]:
model_name = "Mistral-7B-Instruct-v0.1" # Choose a model from the list above
dataset_name = "Math" # Choose a dataset from the list above

tokenizer = AutoTokenizer.from_pretrained(models[model_name], cache_dir='./cache', device_map=torch_device)
model = AutoModelForCausalLM.from_pretrained(models[model_name], cache_dir='./cache', torch_dtype=torch.float16, device_map=torch_device)
tokenizer.pad_token = tokenizer.eos_token # Run depending on whether or not the tokenizer has a built in padding token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# 2. Define the LoRA configuration
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target attention layers
    lora_dropout=0.1,  # Dropout probability
    bias="none"  # Don't train biases
)

In [7]:
# Wrap the model with LoRA
model = get_peft_model(model, lora_config)

In [211]:
# 3. Prepare the dataset
ds = load_dataset(datasets[dataset_name], cache_dir='./cache')

ds = ds['train'].to_pandas()

# Split the dataset
split = 0.9
split_idx = int(len(ds) * split)
train_data_raw = ds[:split_idx]
eval_data_raw = ds[split_idx:]

# Preprocess the dataset
train_data = []
eval_data = []
for i in range(len(train_data_raw)):
    train_data.append({"prompt": train_data_raw.iloc[i]['query'], "answer": train_data_raw.iloc[i]['response']})
for i in range(len(eval_data_raw)):
    eval_data.append({"prompt": eval_data_raw.iloc[i]['query'], "answer": eval_data_raw.iloc[i]['response']})

In [212]:
# Dummy data (This is the right format for the data)

# train_data = [
#     {"prompt": "What is the capital of France?", "answer": "Berlin."},
#     {"prompt": "What is 2 + 2?", "answer": "2 + 2 equals 3."},
# ]
# eval_data = [
#     {"prompt": "What is the capital of Germany?", "answer": "The capital of Germany is Berlin."},
#     {"prompt": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
# ]

In [213]:
train_data = train_data[:2000]
eval_data = eval_data[:200]

In [214]:
def preprocess_function(example):
    prompt = example["prompt"]
    answer = example["answer"]
    tokenized = tokenizer(
        prompt,
        answer,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

In [215]:
train_dataset = Dataset.from_list(train_data).map(preprocess_function, batched=True)
eval_dataset = Dataset.from_list(eval_data).map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [216]:
# 4. Training configuration
training_args = TrainingArguments(
    output_dir=f"./lora_model-{model_name}_dataset-{dataset_name}_v1", # Output directory for the model and predictions
    evaluation_strategy="steps", # Evaluate every 500 steps
    save_strategy="steps", # Save every 500 steps
    save_steps=500,
    per_device_train_batch_size=2, # Batch size per GPU
    gradient_accumulation_steps=4, # Accumulate gradients
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True, # Use mixed precision
    logging_strategy="steps", # Log every 100 steps
    logging_dir="./logs", # Logs
    logging_steps=100, # Log every 100 steps
    save_total_limit=2, # Save only the last 2 checkpoints
    report_to="tensorboard", # Don't report to Hugging Face
)

/zhome/98/5/187238/LoRA_PEFT/lib64/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [219]:
# 5. Trainer setup
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    tokenizer=tokenizer
)

/tmp/ipykernel_1822240/3873853984.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [220]:
# 6. Fine-tune the model
trainer.train()

# File path where you want to save the log
log_file_path = f"./cache/{training_args.output_dir}/log_history.json"

# Save log history to a file
with open(log_file_path, "w") as log_file:
    json.dump(trainer.state.log_history, log_file, indent=4)

Step,Training Loss,Validation Loss
100,0.237200,No log
200,0.234100,No log


In [221]:
# 7. Save the fine-tuned LoRA model
model.save_pretrained(f"./cache/lora_model-{model_name}_dataset-{dataset_name}_v1")
tokenizer.save_pretrained(f"./cache/lora_model-{model_name}_dataset-{dataset_name}_v1")

('./cache/lora_model-Mistral-7B-Instruct-v0.1_dataset-Math_v1/tokenizer_config.json',
 './cache/lora_model-Mistral-7B-Instruct-v0.1_dataset-Math_v1/special_tokens_map.json',
 './cache/lora_model-Mistral-7B-Instruct-v0.1_dataset-Math_v1/tokenizer.model',
 './cache/lora_model-Mistral-7B-Instruct-v0.1_dataset-Math_v1/added_tokens.json',
 './cache/lora_model-Mistral-7B-Instruct-v0.1_dataset-Math_v1/tokenizer.json')

In [210]:
prompt = """Question: Which of the following sentences has the most similar meaning to the sentence below?

Original Sentence:
"Despite the heavy rain, the concert continued as planned."

Options:
1. The concert was canceled due to heavy rain.
2. Heavy rain interrupted the concert midway.
3. The concert went on even though it rained heavily.
4. No rain was forecast, so the concert went on smoothly."""

add_on = """

The correct answer is option: 
"""

model_inputs = tokenizer([prompt+add_on], return_tensors="pt").to(torch_device)
model.to(torch_device)

generated_ids = model.generate(**model_inputs, max_new_tokens=3, do_sample=True)
response = tokenizer.batch_decode(generated_ids)[0]
# print(response)

answer = None
for elem in response[len(prompt+add_on):]:
    if elem.isnumeric():
        answer = int(elem)
        break

if answer is not None:
    print(f"Answer: {answer}")
else:
    print("Answer not found")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: 3
